## Fasta Files - looking for repetitive elements

### Function that reads a fastaq file:

In [33]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [38]:
virus = readFastq('/Users/demol/Downloads/lambda_virus.fa')

In [51]:
list1 = virus[0]
list2 = virus[1]

In [55]:
virus12 = list1 + list2

### Bases counter

In [59]:
import collections
count = collections.Counter()
for seq in virus12:
    count.update(seq)
print(count)

Counter({'G': 6376, 'A': 6293, 'T': 5928, 'C': 5685})


### Genome read from fasta file

In [61]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [63]:
vgenome = readGenome('/Users/demol/Downloads/lambda_virus.fa')

In [64]:
count1 = collections.Counter()
for i in vgenome:
    count1.update(i)
print(count1)

Counter({'G': 12820, 'A': 12334, 'T': 11986, 'C': 11362})


### Function to print reverse complement:

In [71]:
def reverseComplement(s):
    complement = {'A': 'T', 'C': 'G', 'T': 'A', 'G': 'C'}
    t = ''
    for base in s:
        t = complement[base] + t
    return t

In [72]:
r = "ATGTGTACCT"
reverseComplement(r)

'AGGTACACAT'

### Function for naive exact matching algorithm:

In [78]:
def naive(p, t): # p = pattern, t = text
    occurences = []
    for i in range(len(t) - len(p) + 1):  # loop through text t
        match = True
        for j in range(len(p)):         # loop through the pattern p
            if t[i+j] != p[j]:
                match = False
                break
        if match:
            print(i)
            occurences.append(i)        # record the position where is a match
    return(occurences)

In [86]:
t = "AAATTTTGAAATGAATAAA"
p = 'AAA'
naive(p, t)

[0, 8, 16]

In [105]:
def naive_with_rc(p, t):
    occurrences = []
    p_rc = reverseComplement(p)
    for i in range(len(t) - len(p) + 1): # loop over alignments
        match = True

        for j in range(len(p)):  # loop over characters
            if (t[i+j] != p[j]) & (t[i+j] != p_rc[j]):  # compare characters
                match = False
                break
            
        if match:
            occurrences.append(i)# all chars matched; record
    return occurrences

In [114]:
# Example 1
ten_as = "AAAAAAAAAA"
p = 'CGCG'
t = ten_as + "CGCG" + ten_as + 'CGCG' 
naive_with_rc(p, t)

[10, 24]

In [115]:
# Example 2
p = 'CCC'
t = ten_as + "CCC" + ten_as + 'GGG' + ten_as 
naive_with_rc(p, t)

[10, 23]

In [119]:
# Example 3
p = 'AGGT'
t = ten_as + 'AGGT' + ten_as + 'ACCT' + ten_as + 'AGGT' + ten_as + 'ACCT'
print(naive(p, t))
print(naive_with_rc(p, t))

[10, 38]
[10, 24, 38, 52]


__Question 1.__

How many times does __AGGT__ or its reverse complement __ACCT__ occur in the lambda virus genome? E.g. if __AGGT__ occurs 10 times and __ACCT__ occurs 12 times, you should report 22.

In [124]:
p = 'AGGT'
rc = 'ACCT'
print(len(naive_with_rc(p, vgenome)))
print(len(naive(p, vgenome)))
print(len(naive(rc, vgenome)))   # 306

592
150
156


__Question 2.__

How many times does __TTAA__ or its reverse complement occur in the lambda virus genome?

Hint: __TTAA__ and its reverse complement are equal, so remember not to double count.

In [125]:
p = 'TTAA'
print(len(naive_with_rc(p, vgenome)))
print(len(naive(p, vgenome)))

195
195


__Question 3.__

What is the offset of the leftmost occurrence of __ACTAAGT__ or its reverse complement in the Lambda virus genome? E.g. if the leftmost occurrence of __ACTAAGT__ is at offset 40 (0-based) and the leftmost occurrence of its reverse complement __ACTTAGT__ is at offset 29, then report 29.

In [132]:
p = 'ACTAAGT'
occurrences = naive_with_rc(p, vgenome)
p_rc = 'ACTTAGT'
occurences_rc = naive(p_rc, vgenome)
print('offset of leftmost occurrence: %d' % min(occurrences))
print('offset of leftmost occurrence: %d' % min(occurrences_rc))

offset of leftmost occurrence: 26028
offset of leftmost occurrence: 26028


__Question 4.__

What is the offset of the leftmost occurrence of __AGTCGA__ or its reverse complement in the Lambda virus genome?

In [131]:
p = 'AGTCGA'
r = 'TCGACT'
occurrences = naive_with_rc(p, vgenome)
occ_naive_p = naive(p, vgenome)
occ_naive_r = naive(r, vgenome)

print('offset of leftmost occurrence: %d' % min(occurrences)) 
print('offset of leftmost occurrence: %d' % min(occ_naive_p)) 
print('offset of leftmost occurrence: %d' % min(occ_naive_r))   # 450

offset of leftmost occurrence: 49
offset of leftmost occurrence: 18005
offset of leftmost occurrence: 450


__Question 5.__

Sometimes we would like to find approximate matches for P in T. That is, we want to find occurrences with one or more differences.

For Questions 5 and 6, make a new version of the __naive function__ called __naive_2mm__ that allows up to 2 mismatches per occurrence. Unlike for the previous questions, do not consider the reverse complement here. We're looking for approximate matches for P itself, not its reverse complement.


For example, __ACTTTA__ occurs twice in __ACTTACTTGATAAAGT__, once at offset 0 with 2 mismatches, and once at offset 4 with 1 mismatch. So __naive_2mm('ACTTTA', 'ACTTACTTGATAAAGT')__ should return the list __[0, 4]__.


How many times does __TTCAAGCC__ occur in the Lambda virus genome when allowing up to 2 mismatches?

In [217]:
def naive_2mm(p, t): # p = pattern, t = text
    occurences = []
    
    for i in range(len(t) - len(p) + 1):  # loop through text t
        match = True
        count = 0
        for j in range(len(p)):   # loop through the pattern p
                if t[i+j] != p[j]:             
                    count += 1
                    if count > 2:
                        match=False
                        break
        if match:
            #print(i)
            #print (t[i:(i + len(p))])
            occurences.append(i)        # record the position where is a match
    return(occurences)


In [218]:
p = 'ACTTTA'
t = 'ACTTACTTGATAAAGT'
naive_2mm(p, t)

[0, 4]

In [219]:
p = 'CTGT'
ten_as = 'AAAAAAAAAA'
t = ten_as + 'CTGT' + ten_as + 'CTTT' + ten_as + 'CGGG' + ten_as
occurrences = naive_2mm(p, t)
print(occurrences)

[10, 24, 38]


In [222]:
p = 'TTCAAGCC'
print(len(naive_2mm(p, vgenome)))
print('offset of leftmost occurrence: %d' % min(occurrences)) 

191
offset of leftmost occurrence: 10


In [224]:
y = 'AGGAGGTT'
print(len(naive_2mm(y, vgenome)))
print('offset of leftmost occurrence: %d' % min(occurrences)) 

215
offset of leftmost occurrence: 10
